Thie notebook explores using BERT for text classification.  Before starting, change the runtime to GPU: Runtime > Change runtime type > Hardware accelerator: GPU.

First, create a folder named ANLP23_data in your Google drive account, and copy this notebook to that folder, along with `data/convote`, `data/loc` and `data/lmrd` from the Github repo.  Double click on this notebook from your drive account, which will open it in the Google Colab environment.  Begin executing the cells from that environment.

Let's give this notebook access to the data in your ANLP23 folder so we can train and evaluate BERT on the `convote` data.  (Note you are only providing this access to yourself as you execute this notebook.)  You can give Colab notebooks access to other data in the same way (by uploading it first to your Drive account, and then providing access here).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import random
import time

Double-check that this notebook is running on the GPU (this should "Running on cuda").

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cuda


In [5]:
def read_labels(filename):
    labels={}
    with open(filename) as file:
        for line in file:
            cols = line.split("\t")
            label = cols[0]
            if label not in labels:
                labels[label]=len(labels)
    return labels

In [6]:
def read_data(filename, labels, max_data_points=None):
    """
    :param filename: the name of the file
    :return: list of tuple ([word index list], label)
    as input for the forward and backward function
    """
    data = []
    data_labels = []
    with open(filename) as file:
        for line in file:
            cols = line.split("\t")
            label = cols[0]
            text = cols[1]

            data.append(text)
            data_labels.append(labels[label])


    # shuffle the data
    tmp = list(zip(data, data_labels))
    random.shuffle(tmp)
    data, data_labels = zip(*tmp)

    if max_data_points is None:
        return data, data_labels

    return data[:max_data_points], data_labels[:max_data_points]

In [7]:
labels=read_labels("/content/drive/MyDrive/ANLP23_data/convote/train.tsv")
print(labels)

{'D': 0, 'R': 1}


We'll limit the training and dev data to 1,000 data points for this exercise.

In [8]:
train_x, train_y=read_data("/content/drive/MyDrive/ANLP23_data/convote/train.tsv", labels, max_data_points=1000)

In [9]:
dev_x, dev_y=read_data("/content/drive/MyDrive/ANLP23_data/convote/dev.tsv", labels, max_data_points=1000)

In [10]:
def evaluate(model, all_x, all_y):
    model.eval()
    corr = 0.
    total = 0.
    with torch.no_grad():
        idx=0
        for x, y in zip(all_x, all_y):

            idx+=1
            y_preds=model.forward(x)
            for idx, y_pred in enumerate(y_preds):
                prediction=torch.argmax(y_pred)
                if prediction == y[idx]:
                    corr += 1.
                total+=1
    return corr/total

In [11]:
class BERTClassifier(nn.Module):

    def __init__(self, params):
        super().__init__()

        self.model_name=params["model_name"]
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, do_lower_case=params["doLowerCase"], do_basic_tokenize=False)
        self.bert = BertModel.from_pretrained(self.model_name)

        self.num_labels = params["label_length"]

        self.fc = nn.Linear(params["embedding_size"], self.num_labels)

    def get_batches(self, all_x, all_y, batch_size=32, max_toks=256):

        """ Get batches for input x, y data, with data tokenized according to the BERT tokenizer
      (and limited to a maximum number of WordPiece tokens """

        batches_x=[]
        batches_y=[]

        for i in range(0, len(all_x), batch_size):

            current_batch=[]

            x=all_x[i:i+batch_size]

            batch_x = self.tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=max_toks)
            batch_y=all_y[i:i+batch_size]

            batches_x.append(batch_x.to(device))
            batches_y.append(torch.LongTensor(batch_y).to(device))

        return batches_x, batches_y


    def forward(self, batch_x):

        bert_output = self.bert(input_ids=batch_x["input_ids"],
                         attention_mask=batch_x["attention_mask"],
                         token_type_ids=batch_x["token_type_ids"],
                         output_hidden_states=True)

        bert_hidden_states = bert_output['hidden_states']

        # We're going to represent an entire document just by its [CLS] embedding (at position 0)
        out = bert_hidden_states[-1][:,0,:]

        out = self.fc(out)

        return out#.squeeze()

Now let's train BERT on this data.  A few practicalities of this environment: if you encounter an out of memory error:

* Reset the notebook (Runtime > Factory reset runtime) and execute all cells from the beginning.
* If your `max_length` is high, try reducing the `batch_size` in `get_batches` above.

Even on a GPU, BERT can take a long time to train, so you might try experimenting first with smaller `max_data_points` above. before running it on the full training data.

In [12]:
def train_and_evaluate(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=768, doLowerCase=None):

  start_time=time.time()
  bert_model = BERTClassifier(params={"doLowerCase": doLowerCase, "model_name": bert_model_name, "embedding_size":embedding_size, "label_length": len(labels)})
  bert_model.to(device)

  batch_x, batch_y = bert_model.get_batches(train_x, train_y)
  dev_batch_x, dev_batch_y = bert_model.get_batches(dev_x, dev_y)

  optimizer = torch.optim.Adam(bert_model.parameters(), lr=1e-5)
  cross_entropy=nn.CrossEntropyLoss()

  num_epochs=5
  best_dev_acc = 0.

  for epoch in range(num_epochs):
      bert_model.train()

      # Train
      for x, y in tqdm(list(zip(batch_x, batch_y))):
          y_pred = bert_model.forward(x)
          loss = cross_entropy(y_pred.view(-1, bert_model.num_labels), y.view(-1))
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      # Evaluate
      dev_accuracy=evaluate(bert_model, dev_batch_x, dev_batch_y)
      if epoch % 1 == 0:
          print("Epoch %s, dev accuracy: %.3f" % (epoch, dev_accuracy))
          if dev_accuracy > best_dev_acc:
              torch.save(bert_model.state_dict(), model_filename)
              best_dev_acc = dev_accuracy

  bert_model.load_state_dict(torch.load(model_filename))
  print("\nBest Performing Model achieves dev accuracy of : %.3f" % (best_dev_acc))
  print("Time: %.3f seconds ---" % (time.time() - start_time))


In [13]:
train_and_evaluate("bert-base-cased", "convote-bert-base-cased", train_x, train_y, dev_x, dev_y, labels, embedding_size=768, doLowerCase=False)

100%|██████████| 32/32 [00:41<00:00,  1.31s/it]


Epoch 0, dev accuracy: 0.553


100%|██████████| 32/32 [00:38<00:00,  1.22s/it]


Epoch 1, dev accuracy: 0.588


100%|██████████| 32/32 [00:39<00:00,  1.24s/it]


Epoch 2, dev accuracy: 0.607


100%|██████████| 32/32 [00:39<00:00,  1.23s/it]


Epoch 3, dev accuracy: 0.646


100%|██████████| 32/32 [00:39<00:00,  1.23s/it]


Epoch 4, dev accuracy: 0.665

Best Performing Model achieves dev accuracy of : 0.665
Time: 236.782 seconds ---


**Q1**.  Train BERT on your classification data (from the FeatureExploration_TODO.ipynb exercise) by following the steps outlined above.  At this point you have evaluated a number of different classification methods for that data.  Report those development accuracies below.  Be sure to enable a fair comparison by training and evaluating each method on **the same amount of data**.  Provide a one-sentence short description that details the essentials of each method (e.g., major feature classes, etc.), and tell us what data you're running these models on (either one of the three choices above, or your own data).

|Method|Short description|Accuracy|
|--|--|--|
|Majority class|Because 1 is the majority of training data, this model choose 1 as the default answer for all dev data.|0.471|
|Logistic regression (6.classification/FeatureExploration.ipynb)|This logistic regression uses unigram and a few selected adjectives words as features.|0.888|
|BERT|This is a bert-base model, larger and taking longer time to compute than the logistic regression model.|0.842|



In [14]:
np.unique(np.array(train_y), return_counts=True)

(array([0, 1]), array([465, 535]))

In [15]:
np.unique(np.array(dev_y), return_counts=True)

(array([0, 1]), array([127, 130]))

In [16]:
471/(529+471)

0.471

In [17]:
labels=read_labels("/content/drive/MyDrive/ANLP23_data/lmrd/train.tsv")
print(labels)
train_x, train_y=read_data("/content/drive/MyDrive/ANLP23_data/lmrd/train.tsv", labels, max_data_points=1000)
dev_x, dev_y=read_data("/content/drive/MyDrive/ANLP23_data/lmrd/dev.tsv", labels, max_data_points=1000)

{'pos': 0, 'neg': 1}


In [18]:
train_and_evaluate("bert-base-cased", "lmrd-bert-base-cased", train_x, train_y, dev_x, dev_y, labels, embedding_size=768, doLowerCase=False)

100%|██████████| 32/32 [00:40<00:00,  1.26s/it]


Epoch 0, dev accuracy: 0.775


100%|██████████| 32/32 [00:39<00:00,  1.25s/it]


Epoch 1, dev accuracy: 0.864


100%|██████████| 32/32 [00:40<00:00,  1.25s/it]


Epoch 2, dev accuracy: 0.876


100%|██████████| 32/32 [00:39<00:00,  1.25s/it]


Epoch 3, dev accuracy: 0.872


100%|██████████| 32/32 [00:40<00:00,  1.25s/it]


Epoch 4, dev accuracy: 0.842

Best Performing Model achieves dev accuracy of : 0.876
Time: 281.003 seconds ---


**Q2.** As you can see, training `bert-base` can be expensive.  Google has released a number of [smaller BERT models](https://github.com/google-research/bert) with fewer layers (2, 4, 6, 8, 10) and smaller dimensions (128, 256, 512) that effectively trade off accuracy for speed.  Select three of these models and train them; report both their accuracy and training time.



To use these models in the huggingface library that we have been using, the huggingface name of the model can be derived from the URL linking to it:

https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-2_H-128_A-2.zip -> `google/bert_uncased_L-2_H-128_A-2`

All of these smaller models are uncased (so all text is lowercase), so be sure to set `doLowerCase` to be true.  You'll also need to change the `embedding_size` parameter to this function based on the H value from the model (listed both on the BERT Github page and in the model's URL).  One sample model is provided below.

In [19]:
train_and_evaluate("google/bert_uncased_L-2_H-128_A-2", "lmrd-uncased_L-2_H-128_A-2", train_x, train_y, dev_x, dev_y, labels, embedding_size=128, doLowerCase=True)

100%|██████████| 32/32 [00:00<00:00, 53.11it/s]


Epoch 0, dev accuracy: 0.457


100%|██████████| 32/32 [00:00<00:00, 62.22it/s]


Epoch 1, dev accuracy: 0.493


100%|██████████| 32/32 [00:00<00:00, 62.46it/s]


Epoch 2, dev accuracy: 0.531


100%|██████████| 32/32 [00:00<00:00, 62.72it/s]


Epoch 3, dev accuracy: 0.566


100%|██████████| 32/32 [00:00<00:00, 61.89it/s]


Epoch 4, dev accuracy: 0.596

Best Performing Model achieves dev accuracy of : 0.596
Time: 6.794 seconds ---


To turn in:

- Go to `File > Download > Download .ipynb` and save your notebook.
- In your browser, print this page to save as PDF.
- Upload both your .ipynb and .pdf files to bCourses as usual.